In [1]:
import os, sys
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# Para que el notebook vea la raíz del proyecto
ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), ".."))
if ROOT_DIR not in sys.path:
    sys.path.append(ROOT_DIR)

print("ROOT_DIR:", ROOT_DIR)
print("PyTorch:", torch.__version__)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


ModuleNotFoundError: No module named 'torch'

In [ ]:
dataset_path = os.path.join(ROOT_DIR, "data", "iq_dataset.npz")
data = np.load(dataset_path)

print("Claves en el archivo:", data.files)

X_train = data["X_train"]   # (N_train, 2, Nsym)
y_train = data["y_train"]   # (N_train,)
X_val   = data["X_val"]
y_val   = data["y_val"]
mods    = data["modulations"]   # array de strings con nombres de clases

print("X_train:", X_train.shape)
print("y_train:", y_train.shape)
print("X_val  :", X_val.shape)
print("y_val  :", y_val.shape)
print("Clases:", mods)
num_classes = len(mods)


In [ ]:
class IQModDataset(Dataset):
    def __init__(self, X, y):
        # X: numpy (N, 2, Nsym)
        # y: numpy (N,)
        self.X = torch.from_numpy(X).float()
        self.y = torch.from_numpy(y).long()

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

batch_size = 64

train_dataset = IQModDataset(X_train, y_train)
val_dataset   = IQModDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

len(train_dataset), len(val_dataset)


In [ ]:
class IQCNN(nn.Module):
    def __init__(self, num_classes):
        super(IQCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv1d(2, 16, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),      # 256 -> 128
            nn.Conv1d(16, 32, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),      # 128 -> 64
            nn.Conv1d(32, 64, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool1d(2),      # 64 -> 32
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),         # 64 * 32 features
            nn.Linear(64 * 32, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

model = IQCNN(num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print(model)


In [ ]:
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for X_batch, y_batch in loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        _, preds = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (preds == y_batch).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


def evaluate(model, loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for X_batch, y_batch in loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            running_loss += loss.item() * X_batch.size(0)
            _, preds = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (preds == y_batch).sum().item()

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    return epoch_loss, epoch_acc


In [ ]:
num_epochs = 15   # puedes subir/bajar esto
best_val_acc = 0.0
model_path = os.path.join(ROOT_DIR, "data", "iq_cnn_model.pth")

for epoch in range(1, num_epochs + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer, criterion, device)
    val_loss, val_acc = evaluate(model, val_loader, criterion, device)

    print(f"Epoch {epoch:02d}: "
          f"Train Loss={train_loss:.4f}, Train Acc={train_acc*100:.2f}% | "
          f"Val Loss={val_loss:.4f}, Val Acc={val_acc*100:.2f}%")

    # Guardar el mejor modelo
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save({
            "model_state_dict": model.state_dict(),
            "classes": mods,
        }, model_path)
        print(f"  -> Modelo mejorado, guardado en {model_path}")

print(f"Mejor exactitud en validación: {best_val_acc*100:.2f}%")


In [ ]:
checkpoint = torch.load(model_path, map_location=device)
model_loaded = IQCNN(num_classes)
model_loaded.load_state_dict(checkpoint["model_state_dict"])
model_loaded.to(device)
model_loaded.eval()

class_names = [c for c in checkpoint["classes"]]
print("Clases:", class_names)

# Tomamos algunos ejemplos de validación
X_batch, y_batch = next(iter(val_loader))
X_batch = X_batch.to(device)
y_batch = y_batch.to(device)

with torch.no_grad():
    outputs = model_loaded(X_batch)
    probs = torch.softmax(outputs, dim=1)
    conf, preds = torch.max(probs, 1)

for i in range(5):  # mostramos 5 ejemplos
    true_label = class_names[y_batch[i].item()]
    pred_label = class_names[preds[i].item()]
    print(f"Ejemplo {i}: true={true_label}, pred={pred_label}, conf={conf[i].item():.2f}")
